<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/Investidor%C3%A913.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px

# --- 1. BASE DE DADOS ENRIQUECIDA ---
# Adicione ou altere os dados das cidades aqui.
def carregar_dados_sp_enriquecido():
    """
    Cria um DataFrame com dados enriquecidos de cidades em São Paulo.
    VOCÊ PODE E DEVE ALTERAR OS VALORES ABAIXO COM DADOS DA SUA PESQUISA.
    """
    data = {
        "Cidade": ["Jundiaí", "Campinas", "Sorocaba", "São José dos Campos", "Santos", "Vinhedo"],
        # Dados Econômicos
        "PIB": [57.7, 72.9, 43.5, 45.9, 31.8, 15.2],  # PIB em R$ bilhões
        "Crescimento_Estimado": [4.5, 4.2, 4.0, 3.8, 3.5, 4.8],  # Crescimento anual estimado (%)
        # Setores Relevantes (1 = Sim, 0 = Não)
        "Setor_Logistica_Industria": [1, 1, 1, 1, 1, 1],
        "Setor_Tecnologia_Servicos": [1, 1, 0, 1, 0, 0],
        # Custos Operacionais (MENOR É MELHOR)
        "Custo_M2_Comercial": [45, 65, 40, 55, 70, 50], # Valor médio do aluguel (R$/m²)
        "Salario_Medio_Setor": [3500, 4200, 3200, 4500, 3800, 3600], # Salário médio (R$) para uma função chave
        "Aliquota_ISS": [3.5, 4.0, 3.0, 5.0, 5.0, 3.0], # Imposto sobre Serviços (%)
        # Incentivos
        "Incentivos_Fiscais": [
            "ProJundiaí (Incentivos Locais)",
            "Polo Tecnológico (Lei de Inovação)",
            "Parque Tecnológico (Incentivos)",
            "Cluster Aeroespacial (Incentivos)",
            "Regime Aduaneiro (Porto)",
            "Incentivos para novas indústrias"
        ]
    }
    df = pd.DataFrame(data)
    return df

# --- 2. LÓGICA DE RECOMENDAÇÃO POR PONTUAÇÃO ---

def calcular_pontuacao(df, perfil_investidor):
    """
    Calcula uma pontuação de compatibilidade para cada cidade com base nos pesos do investidor.
    """
    df_norm = df.copy()

    # Normalização: Transforma todas as métricas em uma escala de 0 a 10
    # Para métricas onde MAIS é MELHOR (ex: Crescimento)
    for metrica in ["Crescimento_Estimado", "Setor_Logistica_Industria", "Setor_Tecnologia_Servicos"]:
        min_val = df_norm[metrica].min()
        max_val = df_norm[metrica].max()
        if (max_val - min_val) > 0:
            df_norm[metrica] = 10 * (df_norm[metrica] - min_val) / (max_val - min_val)
        else:
            df_norm[metrica] = 10 # Se todos os valores forem iguais

    # Para métricas onde MENOS é MELHOR (ex: Custos) - a lógica é invertida
    for metrica in ["Custo_M2_Comercial", "Salario_Medio_Setor", "Aliquota_ISS"]:
        min_val = df_norm[metrica].min()
        max_val = df_norm[metrica].max()
        if (max_val - min_val) > 0:
            # Inverte a pontuação: o menor custo recebe a maior nota (10)
            df_norm[metrica] = 10 * (max_val - df_norm[metrica]) / (max_val - min_val)
        else:
             df_norm[metrica] = 10

    # Cálculo da Pontuação Ponderada
    pontuacao = (
        df_norm['Crescimento_Estimado'] * perfil_investidor['peso_crescimento'] +
        df_norm['Custo_M2_Comercial'] * perfil_investidor['peso_custo_m2'] +
        df_norm['Salario_Medio_Setor'] * perfil_investidor['peso_salario'] +
        df_norm['Aliquota_ISS'] * perfil_investidor['peso_iss'] +
        df_norm['Setor_Logistica_Industria'] * perfil_investidor['peso_logistica'] +
        df_norm['Setor_Tecnologia_Servicos'] * perfil_investidor['peso_tecnologia']
    )

    df['Pontuacao'] = pontuacao
    return df.sort_values(by="Pontuacao", ascending=False)


# --- 3. VISUALIZAÇÃO ---

def plotar_grafico(df_recomendadas, metrica, titulo):
    """Gera um gráfico de barras interativo para qualquer métrica."""
    fig = px.bar(
        df_recomendadas,
        x="Cidade",
        y=metrica,
        text=metrica,
        title=titulo,
        color="Cidade",
    )
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(showlegend=False)
    fig.show()

# --- 4. FUNÇÃO PRINCIPAL ---

def main():
    # Carregar dados
    df_sp = carregar_dados_sp_enriquecido()

    # =============================================================================
    # DEFINA O PERFIL DO SEU INVESTIDOR AQUI!
    # Dê um peso de 0 (não importa) a 10 (muito importante) para cada fator.
    # =============================================================================
    perfil_investidor_1 = {
        "nome": "Indústria Focada em Custo Baixo",
        "peso_crescimento": 6,
        "peso_custo_m2": 9,      # Muito importante ter aluguel barato
        "peso_salario": 8,       # Muito importante ter mão de obra acessível
        "peso_iss": 10,          # MUITO importante ter imposto baixo
        "peso_logistica": 7,     # Importante ser um polo de logística/indústria
        "peso_tecnologia": 2,    # Pouco importante ser um polo de tecnologia
    }

    perfil_investidor_2 = {
        "nome": "Startup de Tecnologia (Fintech)",
        "peso_crescimento": 9,
        "peso_custo_m2": 5,
        "peso_salario": 4,       # Disposto a pagar mais por talento qualificado
        "peso_iss": 6,
        "peso_logistica": 3,
        "peso_tecnologia": 10,   # ESSENCIAL ser um polo de tecnologia
    }

    # --- Executando para o Perfil 1 ---
    print("="*70)
    print(f"Buscando melhores cidades para o perfil: '{perfil_investidor_1['nome']}'")
    print("="*70)

    cidades_recomendadas_1 = calcular_pontuacao(df_sp, perfil_investidor_1)

    print("\nCidades Recomendadas (Ranking):\n")
    # Selecionando colunas para uma visualização limpa
    colunas_display = ["Cidade", "Pontuacao", "Aliquota_ISS", "Custo_M2_Comercial", "Crescimento_Estimado"]
    print(cidades_recomendadas_1[colunas_display].round(1).to_string(index=False))

    # Visualizar a pontuação final
    plotar_grafico(cidades_recomendadas_1.head(3), "Pontuacao", "Pontuação Final das Cidades Recomendadas")

    # --- Executando para o Perfil 2 ---
    print("\n" + "="*70)
    print(f"Buscando melhores cidades para o perfil: '{perfil_investidor_2['nome']}'")
    print("="*70)

    cidades_recomendadas_2 = calcular_pontuacao(df_sp, perfil_investidor_2)

    print("\nCidades Recomendadas (Ranking):\n")
    print(cidades_recomendadas_2[colunas_display].round(1).to_string(index=False))

    plotar_grafico(cidades_recomendadas_2.head(3), "Pontuacao", "Pontuação Final das Cidades Recomendadas")


# Executar o programa
if __name__ == "__main__":
    main()

Buscando melhores cidades para o perfil: 'Indústria Focada em Custo Baixo'

Cidades Recomendadas (Ranking):

             Cidade  Pontuacao  Aliquota_ISS  Custo_M2_Comercial  Crescimento_Estimado
           Sorocaba      363.1           3.0                  40                   4.0
            Jundiaí      347.7           3.5                  45                   4.5
            Vinhedo      345.4           3.0                  50                   4.8
           Campinas      205.8           4.0                  65                   4.2
São José dos Campos      148.8           5.0                  55                   3.8
             Santos      113.1           5.0                  70                   3.5



Buscando melhores cidades para o perfil: 'Startup de Tecnologia (Fintech)'

Cidades Recomendadas (Ranking):

             Cidade  Pontuacao  Aliquota_ISS  Custo_M2_Comercial  Crescimento_Estimado
            Jundiaí      316.7           3.5                  45                   4.5
            Vinhedo      241.0           3.0                  50                   4.8
           Campinas      226.0           4.0                  65                   4.2
           Sorocaba      214.6           3.0                  40                   4.0
São José dos Campos      175.8           5.0                  55                   3.8
             Santos       51.5           5.0                  70                   3.5
